In [10]:
import pandas as pd

In [11]:
# Import Templates
map_house = pd.read_csv('Flourish_templates/map_house.csv', encoding='utf_8_sig', dtype=object)
map_senate = pd.read_csv('Flourish_templates/map_senate.csv', encoding='utf_8_sig', dtype=object)
bar_house = pd.read_csv('Flourish_templates/bar_house.csv', encoding='utf_8_sig', dtype=object)
bar_senate = pd.read_csv('Flourish_templates/bar_senate.csv', encoding='utf_8_sig', dtype=object)

In [29]:
# Make Dummy data
import json
result = json.load(open('APAPI_test/electionResults_senate.json'))
for race in result['races']:
    race['reportingUnits'][0]['candidates'] = [c for c in race['reportingUnits'][0]['candidates'] if 'winner' in c.keys() and c['winner']=='X']
races = result['races']
winners = pd.DataFrame()
for race in races:
    df = pd.DataFrame(race['reportingUnits'][0]['candidates'])
    df['statePostal'] = race['reportingUnits'][0]['statePostal']
    df['stateName'] = race['reportingUnits'][0]['stateName']
    df['raceID'] = race['raceID']
    if 'seatNum' in race.keys():
        df['seatNum'] = race['seatNum']
    winners = pd.concat([winners, df])
    
if len(winners) == 0:
    pd.DataFrame(columns=['first', 'last', 'abbrv', 'party', 'incumbent', 'candidateID', 'polID',
       'ballotOrder', 'polNum', 'voteCount', 'electWon', 'winner',
       'winnerDateTime', 'statePostal', 'stateName', 'raceID', 'seatNum']).to_csv('APAPI/electionResultsWinners_senate.csv', encoding='utf_8_sig', index=False)
else:
    winners.to_csv('APAPI/electionResultsWinners_senate.csv', encoding='utf_8_sig', index=False)
    
# Make Dummy data
import json
result = json.load(open('APAPI_test/electionResults_house.json'))
for race in result['races']:
    race['reportingUnits'][0]['candidates'] = [c for c in race['reportingUnits'][0]['candidates'] if 'winner' in c.keys() and c['winner']=='X']
races = result['races']

winners = pd.DataFrame()
for race in races:
    df = pd.DataFrame(race['reportingUnits'][0]['candidates'])
    df['statePostal'] = race['reportingUnits'][0]['statePostal']
    df['stateName'] = race['reportingUnits'][0]['stateName']
    df['raceID'] = race['raceID']
    if 'seatNum' in race.keys():
        df['seatNum'] = race['seatNum']
    winners = pd.concat([winners, df])
if len(winners) == 0:
    pd.DataFrame(columns=['first', 'last', 'party', 'incumbent', 'candidateID', 'polID',
       'ballotOrder', 'polNum', 'voteCount', 'electWon', 'winner',
       'winnerDateTime', 'statePostal', 'stateName', 'raceID', 'seatNum',
       'abbrv', 'middle']).to_csv('APAPI/electionResultsWinners_house.csv', encoding='utf_8_sig', index=False)
else:
    winners.to_csv('APAPI/electionResultsWinners_house.csv', encoding='utf_8_sig', index=False)

In [48]:
# Import winners data
house = pd.read_csv('APAPI/electionResultsWinners_house.csv', encoding='utf_8_sig', dtype=object)
senate = pd.read_csv('APAPI/electionResultsWinners_senate.csv', encoding='utf_8_sig', dtype=object)

In [43]:
# Import race ids
raceID_house = pd.read_csv('raceID_house.csv', encoding='utf_8_sig', dtype=object)['raceID'].to_list()
raceID_senate = pd.read_csv('raceID_senate.csv', encoding='utf_8_sig', dtype=object)['raceID'].to_list()
house = house[house.raceID.isin(raceID_house)].copy()
senate = senate[senate.raceID.isin(raceID_senate)].copy()

In [50]:
map_house.head(1)

,geometry,districtID,政党,district
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-179....",AK01,残り,アラスカ(01)


In [51]:
house.head(1)

,first,last,party,incumbent,candidateID,polID,ballotOrder,polNum,voteCount,electWon,winner,winnerDateTime,statePostal,stateName,raceID,seatNum,abbrv,middle
0,Jerry,Carl,GOP,True,5819,68313,1.0,3790,138022.0,0.0,X,2022-11-02T21:06:23.002Z,AL,Alabama,1005,1,NaN,NaN


In [52]:
# Make map_house
if len(house) > 0:
    _dict = {'Dem':'民主党', 'GOP':'共和党'}
    _dict2 = map_house.set_index('districtID')['district'].to_dict()
    df = house.copy()
    df['districtID'] = df['statePostal'].astype(str) + df['seatNum'].astype(str).str.zfill(2)
    df['政党'] = df['party'].apply(lambda x: _dict[x] if x in _dict.keys() else 'その他')
    map_house = pd.merge(map_house[['geometry', 'districtID']], df[['districtID', '政党']], on='districtID', how='left').fillna('残り')
    map_house['district'] = map_house['districtID'].apply(lambda x: _dict2[x])
map_house.to_csv('Flourish/map_house.csv', encoding='utf_8_sig', index=False)

In [57]:
# Make map_senate
if len(senate) > 0:
    for i, row in map_senate[map_senate['議席']=='改選'].iterrows():
        t_df = senate[senate.raceID==row.raceID].copy()
        if len(t_df) == 1:
            map_senate.loc[i, '結果'] = _dict[t_df.iloc[0].party] if t_df.iloc[0].party in _dict.keys() else 'その他'
map_senate.to_csv('Flourish/map_senate.csv', encoding='utf_8_sig', index=False)

In [60]:
# Make bar_house
if len(house) > 0:
    _dict = {'Dem':'民主党', 'GOP':'共和党'}
    df = house.copy()
    df.party = df.party.apply(lambda x: _dict[x] if x in _dict.keys() else 'その他')
    result = df.party.value_counts().to_dict()
    for c in ['民主党', 'その他', '共和党']:
        if c not in result.keys():
            result[c] = 0
    for c in ['民主党', 'その他', '共和党']:
        bar_house.iloc[0][c] = result[c]
    bar_house.iloc[0]['残り'] = 435 - sum(result.values())
bar_house.to_csv('Flourish/bar_house.csv', encoding='utf_8_sig', index=False)

In [61]:
# Make bar_senate
if len(senate) > 0:
    _dict = {'Dem':'民主党', 'GOP':'共和党'}
    df = senate.copy()
    df.party = df.party.apply(lambda x: _dict[x] if x in _dict.keys() else 'その他')
    result = df.party.value_counts().to_dict()
    for c in ['民主党', 'その他', '共和党']:
        if c not in result.keys():
            result[c] = 0
    for c in ['民主党', 'その他', '共和党']:
        bar_senate.iloc[0][c] = result[c]
    bar_senate.iloc[0]['残り'] = 35 - sum(result.values())
bar_senate.to_csv('Flourish/bar_senate.csv', encoding='utf_8_sig', index=False)